## Задание № 1

Напишите функцию, которая возвращает название валюты (поле 'Name') с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru.

## Задание № 2

Добавьте в класс Rate параметр diff(со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению с прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [36]:
import requests
class Rate:
    def __init__(self, format_='value',diff_ = False):
        self.format = format_
        self.diff = diff_
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'ID': 'R01239',
            'NumCode': '978',
            'CharCode': 'EUR',
            'Nominal': 1,
            'Name': 'Евро',
            'Value': 69.226,
            'Previous': 68.7775
        }
        
        - изменение курса валюты по сравнению с прошлым значением при self.format = 'value' и diff = True:
         Rate('value', diff_ = True).make_format('EUR')
        0.4919000000000011
        
        курс данной валюты сейчас при self.format = 'value' и diff_ = False:
        Rate('value').make_format('EUR')
        62.8299
        
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                
                if self.diff:
                    return response[currency]['Value']-response[currency]['Previous']
                else:
                    return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')
    
    def maximum_course_value(self, currency):
        
        """
        Возвращает информацию о валюте currency:
        - возвращает название валюты (поле 'Name') с максимальным значением курса self.format = 'name':
        
        Rate('name').make_format('EUR')
        (Евро, 69.226)
        """
        
        course_value = self.exchange_rates()
        
        if currency in course_value:
            
            if self.format == 'name':
                
                if course_value[currency]['Value'] >= course_value[currency]['Previous']:
                    max_value = course_value[currency]['Value']
                else:
                    max_value = course_value[currency]['Previous']
                
                return (course_value[currency]['Name'], max_value)
        
        return 'Error'

In [44]:
r = Rate('name').maximum_course_value('EUR')
print(r)
r = Rate(format_='value')
print(r.usd())
r = Rate(format_='value', diff_ = True)
print(r.usd())

('Евро', 69.226)
62.8299
0.4919000000000011


## Задание № 3

Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

In [34]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [35]:
class Designer(Employee):
    def __init__(self, name, seniority,awards):
        super().__init__(name, seniority)
        self.awards = awards
        
        """Проверяем минимальное количество премий"""
        if awards < 2:
            print(str(self.name)+' имеет мало премий!')
            self.awards = 2
            
        self.seniority = self.awards*2
        
    def awards_up(self):
        """Увеличивает количество премий"""
        self.awards += 1
        self.seniority +=2
        
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print('Имя сотрудника: '+str(self.name)+', должность сотрудника: '+str(self.grade)+', премии сотрудника: '+str(self.awards))   
        
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 7 == 0:
            self.grade_up()
         
        # условие увеличения премий сотрудника
        if self.seniority % 14 == 0:
            self.awards_up()
        
        # публикация результатов
        return self.publish_grade()

In [36]:
alex = Designer('Александр', 0, 1)
elena = Designer('Елена', 0, 2)
Max = Designer('Максим', 0, 3)
for i in range(20):
    alex.check_if_it_is_time_for_upgrade()
    elena.check_if_it_is_time_for_upgrade()
    Max.check_if_it_is_time_for_upgrade()
    print(' ')

Александр имеет мало премий!
Имя сотрудника: Александр, должность сотрудника: 1, премии сотрудника: 2
Имя сотрудника: Елена, должность сотрудника: 1, премии сотрудника: 2
Имя сотрудника: Максим, должность сотрудника: 2, премии сотрудника: 3
 
Имя сотрудника: Александр, должность сотрудника: 1, премии сотрудника: 2
Имя сотрудника: Елена, должность сотрудника: 1, премии сотрудника: 2
Имя сотрудника: Максим, должность сотрудника: 2, премии сотрудника: 3
 
Имя сотрудника: Александр, должность сотрудника: 2, премии сотрудника: 2
Имя сотрудника: Елена, должность сотрудника: 2, премии сотрудника: 2
Имя сотрудника: Максим, должность сотрудника: 2, премии сотрудника: 3
 
Имя сотрудника: Александр, должность сотрудника: 2, премии сотрудника: 2
Имя сотрудника: Елена, должность сотрудника: 2, премии сотрудника: 2
Имя сотрудника: Максим, должность сотрудника: 2, премии сотрудника: 3
 
Имя сотрудника: Александр, должность сотрудника: 2, премии сотрудника: 2
Имя сотрудника: Елена, должность сотрудник